# Aula extra - Como analisar os dados de rentabilidade?

### Desafio:

* Construir um código que faça um ranking dos melhores fundos em três períodos de rentabilidade diferentes. Iremos selecionar os 10 melhores posicionados no quesito rentabilidade no curto e no longo prazo.

* Essa aula será mais curta. 

### Passo a passo:

   **Passo 1** - Puxar as tabelas igual na aula 2.
   
   **Passo 2** - Escolher quais períodos de rentabilidade iremos analisar.
   
   **Passo 3** - Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

   **Passo 4** - Transformar as rentabilidades em números decimais, ao invés de string. 
   
   **Passo 5** - Juntar as tabelas, igual na aula 2. 
   
   **Passo 6** - Filtrar os ETFs alavancados. 
   
   **Passo 7** - Fazer os rankings de rentabilidade por período e soma-los. 
   
   **Passo 8** - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

# Passo 1: Puxar as tabelas igual na aula 2.

In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [62]:
driver = webdriver.Chrome(service = Service(ChromeDriverManager().install()))

url = "https://www.etf.com/etfanalytics/etf-finder"

driver.get(url)

time.sleep(5)

botao_100 = driver.find_element("xpath", '''html/body/div[5]/section/div/div[3]/section/div
                                                /div/div/div/div[2]/
                                section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", botao_100)

numero_paginas = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/
section/div/div/div/div/div[2]/section[2]/div[2]/section[2]/div[2]/div/label[2]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_completa = pd.concat(lista_de_tabela_por_pagina)

botao_aba = driver.find_element("xpath", ''' /html/body/div[5]/section/div/div[3]/section/div/div/div/div/
div[2]/section[2]/div[2]/ul/li[2]/span''')

driver.execute_script("arguments[0].click();", botao_aba)

#voltando a 1 página

for pagina in range(0, numero_paginas):
    
    botao_voltar_pagina = driver.find_element("xpath", '//*[@id="previousPage"]')
    
    driver.execute_script("arguments[0].click();", botao_voltar_pagina)
    
    
lista_de_tabela_por_pagina = []

for pagina in range(0, numero_paginas):
    
    tabela = driver.find_element("xpath", '''/html/body/div[5]/section/div/div[3]/section/div/
                                    div/div/div/div[2]/section[2]/div[2]/div/table''')

    html_tabela = tabela.get_attribute("outerHTML")

    tabela_final = pd.read_html(html_tabela)[0]
    
    lista_de_tabela_por_pagina.append(tabela_final)
    
    botao_avancar_pagina = driver.find_element("xpath", '//*[@id="nextPage"]')
    
    driver.execute_script("arguments[0].click();", botao_avancar_pagina)
    

base_de_dados_performance = pd.concat(lista_de_tabela_por_pagina)

driver.quit()

In [63]:
base_de_dados_completa = base_de_dados_completa.set_index('Ticker')
base_de_dados_performance = base_de_dados_performance.set_index("Ticker")

In [64]:
base_de_dados_performance

,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
Ticker,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,-3.91%,-2.65%,3.22%,-6.70%,11.81%,9.95%,12.08%,03/01/23
IVV,iShares Core S&P 500 ETF,-3.91%,-2.66%,3.21%,-6.71%,12.03%,9.99%,12.13%,03/01/23
VTI,Vanguard Total Stock Market ETF,-3.34%,-1.57%,4.61%,-6.63%,11.66%,9.65%,11.82%,03/01/23
VOO,Vanguard S&P 500 ETF,-3.91%,-2.68%,3.26%,-6.71%,11.87%,9.98%,12.14%,03/01/23
QQQ,Invesco QQQ Trust,-3.23%,-0.62%,9.36%,-14.10%,12.97%,12.91%,16.85%,03/01/23
...,...,...,...,...,...,...,...,...,...
CVLC,Calvert US Large-Cap Core Responsible Index ETF,-3.88%,--,--,--,--,--,--,03/01/23
CVMC,Calvert US Mid-Cap Core Responsible Index ETF,-3.78%,--,--,--,--,--,--,03/01/23
CVIE,Calvert International Responsible Index ETF,-3.79%,--,--,--,--,--,--,03/01/23


# Passo 2: Escolher quais períodos de rentabilidade iremos analisar.

No nosso caso, iremos pegar as janelas de 3, 5 e 10 anos. 

In [65]:
base_de_dados_performance = base_de_dados_performance[['3 Years', '5 Years']]

base_de_dados_performance

,3 Years,5 Years
Ticker,,
SPY,11.81%,9.95%
IVV,12.03%,9.99%
VTI,11.66%,9.65%
VOO,11.87%,9.98%
QQQ,12.97%,12.91%
...,...,...
CVLC,--,--
CVMC,--,--
CVIE,--,--


# Passo 3: Retirar os dados faltantes. Fundos que não possuem os períodos necessários, ficarão de fora. 

In [66]:
base_de_dados_performance = base_de_dados_performance.replace("--", pd.NA)

base_de_dados_performance

,3 Years,5 Years
Ticker,,
SPY,11.81%,9.95%
IVV,12.03%,9.99%
VTI,11.66%,9.65%
VOO,11.87%,9.98%
QQQ,12.97%,12.91%
...,...,...
CVLC,NaN,NaN
CVMC,NaN,NaN
CVIE,NaN,NaN


In [67]:
base_de_dados_performance = base_de_dados_performance.dropna()

base_de_dados_performance

,3 Years,5 Years
Ticker,,
SPY,11.81%,9.95%
IVV,12.03%,9.99%
VTI,11.66%,9.65%
VOO,11.87%,9.98%
QQQ,12.97%,12.91%
...,...,...
LD,6.06%,-2.88%
ERUS,-37.41%,-23.12%
LTL,-1.49%,-2.69%


# Passo 4: Transformar as rentabilidades em números decimais

In [68]:
base_de_dados_performance['3 Years'] = (base_de_dados_performance['3 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance['5 Years'] = (base_de_dados_performance['5 Years'].
                                        str.rstrip('%').astype(float)/100)

base_de_dados_performance

C:\Users\rmnf\AppData\Local\Temp\ipykernel_13856\2273226170.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_performance['3 Years'] = (base_de_dados_performance['3 Years'].
C:\Users\rmnf\AppData\Local\Temp\ipykernel_13856\2273226170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_de_dados_performance['5 Years'] = (base_de_dados_performance['5 Years'].


,3 Years,5 Years
Ticker,,
SPY,0.1181,0.0995
IVV,0.1203,0.0999
VTI,0.1166,0.0965
VOO,0.1187,0.0998
QQQ,0.1297,0.1291
...,...,...
LD,0.0606,-0.0288
ERUS,-0.3741,-0.2312
LTL,-0.0149,-0.0269


# Passo 5: Juntar as tabelas, igual na aula 2.

In [69]:
base_de_dados_final = base_de_dados_completa.join(base_de_dados_performance, how = "inner")

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years
Ticker,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$359.90B,0.1181,0.0995
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$297.49B,0.1203,0.0999
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$275.63B,0.1166,0.0965
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$274.28B,0.1187,0.0998
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$156.34B,0.1297,0.1291
...,...,...,...,...,...,...,...
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.24M,0.0606,-0.0288
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.3741,-0.2312
LTL,ProShares Ultra Telecommunications,Leveraged Equity: U.S. Telecoms,ProShares,0.95%,$649.08K,-0.0149,-0.0269


# Passo 6: Filtrar os ETFs alavancados.

In [70]:
base_de_dados_final = base_de_dados_final[~base_de_dados_final['Segment'].str.contains("Leveraged")]
base_de_dados_final = base_de_dados_final[base_de_dados_final['Segment'].str.contains("China")]

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years
Ticker,,,,,,,
MCHI,iShares MSCI China ETF,Equity: China - Total Market,Blackrock,0.58%,$8.31B,-0.0586,-0.0500
KWEB,KraneShares CSI China Internet ETF,Equity: China Internet,CICC,0.69%,$6.05B,-0.1233,-0.1154
FXI,iShares Trust - China Large-Cap ETF,Equity: China - Large Cap,Blackrock,0.74%,$5.48B,-0.0862,-0.0710
ASHR,Xtrackers Harvest CSI 300 China A-Shares ETF,Equity: China - Total Market,DWS,0.65%,$2.74B,0.0278,0.0027
GXC,SPDR S&P China ETF,Equity: China - Total Market,State Street Global Advisors,0.59%,$1.21B,-0.0440,-0.0434
CQQQ,Invesco China Technology ETF,Equity: China Information Technology,Invesco,0.70%,$953.40M,-0.0568,-0.0589
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$764.95M,-0.0420,-0.0339
KBA,KraneShares Bosera MSCI China A 50 Connect Ind...,Equity: China - Total Market,CICC,0.56%,$469.11M,0.0494,0.0194
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$346.06M,0.0525,0.0244


# Passo 7: Fazer os rankings de rentabilidade por período e soma-los. 

In [71]:
base_de_dados_final['rank_3_anos'] = base_de_dados_final['3 Years'].rank(ascending = False)
base_de_dados_final['rank_5_anos'] = base_de_dados_final['5 Years'].rank(ascending = False)
base_de_dados_final['rank_10_anos'] = base_de_dados_final['10 Years'].rank(ascending = False)

base_de_dados_final

KeyError: '10 Years'

In [ ]:
base_de_dados_final['rank_final'] =  (base_de_dados_final['rank_5_anos'] + 
                                      base_de_dados_final['rank_10_anos']) 

base_de_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,
MCHI,iShares MSCI China ETF,Equity: China - Total Market,Blackrock,0.58%,$8.31B,-0.0586,-0.0500,28.0,27.0,55.0
KWEB,KraneShares CSI China Internet ETF,Equity: China Internet,CICC,0.69%,$6.05B,-0.1233,-0.1154,32.0,31.0,63.0
FXI,iShares Trust - China Large-Cap ETF,Equity: China - Large Cap,Blackrock,0.74%,$5.48B,-0.0862,-0.0710,29.0,29.0,58.0
ASHR,Xtrackers Harvest CSI 300 China A-Shares ETF,Equity: China - Total Market,DWS,0.65%,$2.74B,0.0278,0.0027,13.0,14.0,27.0
GXC,SPDR S&P China ETF,Equity: China - Total Market,State Street Global Advisors,0.59%,$1.21B,-0.0440,-0.0434,25.0,25.0,50.0
CQQQ,Invesco China Technology ETF,Equity: China Information Technology,Invesco,0.70%,$953.40M,-0.0568,-0.0589,27.0,28.0,55.0
CXSE,WisdomTree China ex-State-Owned Enterprises Fund,Equity: China - Total Market,WisdomTree,0.32%,$764.95M,-0.0420,-0.0339,24.0,22.0,46.0
KBA,KraneShares Bosera MSCI China A 50 Connect Ind...,Equity: China - Total Market,CICC,0.56%,$469.11M,0.0494,0.0194,8.0,7.0,15.0
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$346.06M,0.0525,0.0244,7.0,4.0,11.0


# Passo 8: Gran finale - Ordenar do menor ranking pro maior, encontrando os "melhores" ETFs.

In [ ]:
melhores_etfs_china = base_de_dados_final.sort_values(by = "rank_final")

melhores_etfs_china

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,
CHIE,Global X MSCI China Energy ETF,Equity: China Energy,"Mirae Asset Global Investments Co., Ltd.",0.67%,$10.58M,0.2311,0.1027,1.0,1.0,2.0
KGRN,KraneShares MSCI China Clean Technology Index ETF,Equity: China Environment,CICC,0.78%,$93.14M,0.1466,0.0503,2.0,2.0,4.0
CNYA,iShares MSCI China A ETF,Equity: China - Total Market,Blackrock,0.60%,$330.65M,0.0550,0.0260,6.0,3.0,9.0
ASHX,Xtrackers MSCI China A Inclusion Equity ETF,Equity: China - Total Market,DWS,0.60%,$5.72M,0.0556,0.0216,5.0,5.0,10.0
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$346.06M,0.0525,0.0244,7.0,4.0,11.0
CHIM,Global X MSCI China Materials ETF,Equity: China Materials,"Mirae Asset Global Investments Co., Ltd.",0.66%,$3.59M,0.0852,0.0064,3.0,12.0,15.0
KBA,KraneShares Bosera MSCI China A 50 Connect Ind...,Equity: China - Total Market,CICC,0.56%,$469.11M,0.0494,0.0194,8.0,7.0,15.0
KFYP,KraneShares CICC China Leaders 100 Index ETF,Equity: China - Total Market,CICC,0.68%,$5.87M,0.0347,0.0103,10.0,8.0,18.0
CBON,VanEck ChinaAMC China Bond ETF,"Fixed Income: China - Broad Market, Broad-base...",VanEck,0.51%,$47.39M,0.0283,0.0206,12.0,6.0,18.0


In [ ]:
melhores_etfs_china["3 Years"] = melhores_etfs_china["3 Years"]*100
melhores_etfs_china["5 Years"] = melhores_etfs_china["5 Years"]*100
melhores_etfs_china["10 Years"] = melhores_etfs_china["10 Years"]*100

melhores_etfs_china.head()

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,rank_3_anos,rank_5_anos,rank_final
Ticker,,,,,,,,,,
CHIE,Global X MSCI China Energy ETF,Equity: China Energy,"Mirae Asset Global Investments Co., Ltd.",0.67%,$10.58M,23.11,10.27,1.0,1.0,2.0
KGRN,KraneShares MSCI China Clean Technology Index ETF,Equity: China Environment,CICC,0.78%,$93.14M,14.66,5.03,2.0,2.0,4.0
CNYA,iShares MSCI China A ETF,Equity: China - Total Market,Blackrock,0.60%,$330.65M,5.50,2.60,6.0,3.0,9.0
ASHX,Xtrackers MSCI China A Inclusion Equity ETF,Equity: China - Total Market,DWS,0.60%,$5.72M,5.56,2.16,5.0,5.0,10.0
CHIQ,Global X MSCI China Consumer Discretionary ETF,Equity: China Consumer Discretionary,"Mirae Asset Global Investments Co., Ltd.",0.65%,$346.06M,5.25,2.44,7.0,4.0,11.0
